# WS csv파일과 PER csv 파일 join

In [11]:
import pandas as pd
for y in [str(i) for i in range(2003,2021)]:
    print(y)
    per = pd.read_csv('./PER/{}_PER.csv'.format(y), index_col=0).dropna(axis=1)
    per = per[per.AST!='AST']
    per['Player'] = per.PLAYER.apply(lambda x: x.split(',')[0])
    ws = pd.read_csv('./WS/{}_WS.csv'.format(y), index_col=0).dropna(axis=1)
    try:
        df = pd.merge(per, ws, on='Player').drop(['PER_y', 'G'], axis=1)
    except:
        df = pd.merge(per, ws, on='Player').drop('G', axis=1)
    
    COL=list(df.columns)
    for c in COL:
        try:
            df[c]=df[c].apply(lambda x: float(x))
        except:
            continue     
    df1 = df.drop(['Player', 'PLAYER', 'Pos', 'Rk', 'Age'], axis=1).groupby('Tm').mean()
    df2 = df.drop(['Player', 'PLAYER', 'Pos', 'Rk'], axis=1).groupby('Tm').mean()
    df1.to_csv('./data/data_without_age_{}.csv'.format(y))
    df2.to_csv('./data/data_with_age_{}.csv'.format(y))
    

2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


# scores csv파일 전처리

In [104]:
import pandas as pd
import numpy as np
scores = pd.read_csv('NBA_scores.csv')
scores = scores.drop('Unnamed: 0', axis=1)
# result값으로 유효 경기인지 확인
scores = scores[scores.result.apply(lambda x: ',' in str(x))]
# 포스트시즌 경기 제거 공통적으로 하이픈이 존재함
scores = scores[scores.result.apply(lambda x: '-' not in x)]
#result 중 괄호 안의 값들을 제거
scores['result'] = scores.result.apply(lambda x: x.split('(')[0])
# result값으로 홈팀 원정팀 스코어 정보 확인
scores['result'] = scores.result.apply(lambda x: x.split(','))
#Date 연월일로 쪼개기
scores['year'] = scores.DATE.apply(lambda x: int(str(x)[:4]))
scores['month'] = scores.DATE.apply(lambda x: int(str(x)[4:6]))
#02-03시즌 즉 2002년이 아니거나 2002년 10월 이후경기만
scores=scores[(scores.year!=2002)|(scores.month >=10)]
#home팀명(약자) away팀명(약자) home팀 득점 away팀 득점 뽑기 
scores['HOME'] = scores.home.apply(lambda x: x.split()[-1])
scores['AWAY'] = scores.matchup.apply(lambda x: x.split()[-1])
scores['winner'] = scores.result.apply(lambda x: x[0].split()[0])
scores['loser'] = scores.result.apply(lambda x: x[1].split()[0])
#시즌
scores['SEASON'] = scores.year+scores.month.apply(lambda x: 1 if x >= 10 else 0)
#승무패: result의 경우 승리팀이 앞에 오기때문에 홈/원정으로 순서를 바꿔주어야 함
scores['LOSE_SCORE'] = scores.result.apply(lambda x: int(x[1].split()[-1]) ) 
scores['WIN_SCORE'] = scores.result.apply(lambda x: int(x[0].split()[-1]) )
scores['GAME_RESULT'] = scores.WIN_SCORE - scores.LOSE_SCORE #점수차
scores['AwayWIN'] = scores.AWAY == scores.winner
scores['AwayWIN'] = scores.AwayWIN.apply(lambda x: -1 if x==True  else 1)
scores['GAME_RESULT'] = scores.GAME_RESULT*scores.AwayWIN
#저장
scores[['DATE','SEASON','HOME', 'AWAY','GAME_RESULT']].to_csv('SCORE.csv')


scores

,matchup,home,result,winner high,loser high,DATE,year,month,HOME,AWAY,winner,loser,SEASON,LOSE_SCORE,WIN_SCORE,GAME_RESULT,AwayWIN
857,Utah UTAH,Seattle SEA,"[UTAH 89, SEA 82]",NaN,NaN,20021006,2002,10,SEA,UTAH,UTAH,SEA,2003,82,89,-7,-1
858,Milwaukee MIL,Minnesota MIN,"[MIN 88, MIL 82]",NaN,NaN,20021008,2002,10,MIN,MIL,MIN,MIL,2003,82,88,6,1
859,San Antonio SA,Philadelphia PHI,"[SA 95, PHI 82]",NaN,NaN,20021008,2002,10,PHI,SA,SA,PHI,2003,82,95,-13,-1
860,Utah UTAH,Portland POR,"[POR 93, UTAH 80]",NaN,NaN,20021008,2002,10,POR,UTAH,POR,UTAH,2003,80,93,13,1
861,Golden State GS,Seattle SEA,"[GS 84, SEA 75]",NaN,NaN,20021008,2002,10,SEA,GS,GS,SEA,2003,75,84,-9,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27639,Orlando ORL,Memphis MEM,"[ORL 120, MEM 115]",Terrence Ross 24 Pts,Jonas Valanciunas 27 Pts,20200310,2020,3,MEM,ORL,ORL,MEM,2020,115,120,-5,-1
27640,Dallas DAL,San Antonio SA,"[SA 119, DAL 109]",LaMarcus Aldridge 24 Pts,Luka Doncic 38 Pts,20200310,2020,3,SA,DAL,SA,DAL,2020,109,119,10,1
27641,Phoenix PHX,Portland POR,"[POR 121, PHX 105]",Damian Lillard 25 Pts,Devin Booker 29 Pts,20200310,2020,3,POR,PHX,POR,PHX,2020,105,121,16,1
27642,LA LAC,Golden State GS,"[LAC 131, GS 107]",Kawhi Leonard 23 Pts,Dragan Bender 23 Pts,20200310,2020,3,GS,LAC,LAC,GS,2020,107,131,-24,-1


In [106]:
SCORES = pd.read_csv('SCORE.csv').drop('Unnamed: 0', axis=1)
SCORES[SCORES.SEASON <= 2020]

,DATE,SEASON,HOME,AWAY,GAME_RESULT
0,20021006,2003,SEA,UTAH,-7
1,20021008,2003,MIN,MIL,6
2,20021008,2003,PHI,SA,-13
3,20021008,2003,POR,UTAH,13
4,20021008,2003,SEA,GS,-9
...,...,...,...,...,...
25394,20200310,2020,MEM,ORL,-5
25395,20200310,2020,SA,DAL,10
25396,20200310,2020,POR,PHX,16
25397,20200310,2020,GS,LAC,-24


# 스코어정보와 팀의 선수평균 스탯을 조인
##### 예측을 위해 직전시즌의 스탯평균정보와 스코어를 매칭

In [126]:
#17-8시즌까지의 스탯정보 사용 사용
#18-19시즌까지 예측
year = [i for i in range(2003,2019)]
for y in year:
    df1=pd.read_csv('./data/data_with_age_{}.csv'.format(y))
    df2=pd.read_csv('./data/data_without_age_{}.csv'.format(y))
    #나이 포함 
    df_x1 =pd.merge(SCORES[SCORES.SEASON == y+1], df1, left_on='HOME', right_on='Tm')
    df_fin1 = pd.merge(df_x1, df1, left_on='AWAY', right_on='Tm')
    df_fin1.to_csv('./data2/data_with_age_stat{}_score{}.csv'.format(y,y+1))
    #나이 미포함
    df_x2 =pd.merge(SCORES[SCORES.SEASON == y+1], df2, left_on='HOME', right_on='Tm')
    df_fin2 = pd.merge(df_x2, df2, left_on='AWAY', right_on='Tm')
    df_fin2.to_csv('./data2/data_without_age_stat{}_score{}.csv'.format(y,y+1))

# csv파일을 모두 합치고(concat) 문자열 컬럼을 제거해 머신러닝에 쓸수있도록 최종처리한다

In [127]:
import pandas as pd
year = [i for i in range(2003,2019)]
temp1 = pd.DataFrame()
temp2 = pd.DataFrame()
for y in year:
    x = pd.read_csv('./data2/data_with_age_stat{}_score{}.csv'.format(y,y+1))
    y = pd.read_csv('./data2/data_without_age_stat{}_score{}.csv'.format(y,y+1))
    temp1 = pd.concat([temp1, x])
    temp2 = pd.concat([temp2, y])
temp1.to_csv('final_data_with_age.csv')
temp2.to_csv('final_data_without_age.csv')
    

C:\Users\103-04\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\Users\103-04\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':
